# Stance Predictions

In [1]:
import pandas as pd
from collections import Counter
import ast

In [2]:
fn = '../../results-test/stance/chatnoir_10_custom_stopw_lemmas_chatgpt.tsv'
stance_df = pd.read_csv(fn, sep='\t')
stance_df['gpt_stance'] = stance_df['gpt_stance'].apply(ast.literal_eval)
stance_df.head()

,qid,query,docno,score,title_text,html_plain,rank,truncated_html_plain,gpt_stance
0,1,should teacher get tenure,clueweb22-en0031-49-02531,2800.1940,Pro & Con Quotes: Should Teachers Get Tenure? ...,Last updated on: 1/13/2011 | Author: ProCon.or...,0,Last updated on: 1/13/2011 | Author: ProCon.or...,"(NO, NEU, PRO)"
1,1,should teacher get tenure,clueweb22-en0042-24-00769,1911.2747,Teacher Tenure - Pros & Cons - ProCon.org,Last updated on: 1/13/2011 | Author: ProCon.or...,1,Last updated on: 1/13/2011 | Author: ProCon.or...,"(NEU, PRO, NEU)"
2,1,should teacher get tenure,clueweb22-en0011-57-13248,1896.7843,Teacher Tenure Laws | LegalMatch,Most states have laws that protect public scho...,2,Most states have laws that protect public scho...,"(NEU, NO, NO)"
3,1,should teacher get tenure,clueweb22-en0038-04-18313,1813.0868,What Is Teacher Tenure? | Education.com,"Charter schools, merit pay, vouchers, oh my! T...",3,"Charter schools, merit pay, vouchers, oh my! T...","(PRO, PRO, PRO)"
4,1,should teacher get tenure,clueweb22-en0022-88-08363,1807.3335,Tenure | American Federation of Teachers,Tenure\n\nShare This\nPrint\n\nHow Due Process...,4,Tenure\n\nShare This\nPrint\n\nHow Due Process...,"(PRO, PRO, PRO)"


In [3]:
def aggregate_stance(stances:tuple[str]) -> str:
    """
    Aggregate stance labels by the following rules (in that order):
    - majority vote
    - NO if there is a NO
    - NEU if there is a NEU
    """
    assert len(stances) == 3, (stances, len(stances), 'the code below assumes 3 items in the tuple.')
    c = Counter(stances)
    c = {v:k for k,v in c.items()}
    if 3 in c.keys():
        return c[3]
    if 2 in c.keys():
        return c[2]
    if 'NO' in stances:
        return 'NO'
    if 'NEU' in stances:
        return 'NEU'
    raise AssertionError(f'Could not aggregate stances: {stances}')

def aggregate_stance_certain_NO(stances:tuple[str]) -> str:
    """
    Aggregate stance labels by the following rules (in that order):
    - majority vote
    - NEU else

    this is used when NO is used to identify quality ratings
    """
    assert len(stances) == 3, (stances, len(stances), 'the code below assumes 3 items in the tuple.')
    c = Counter(stances)
    c = {v:k for k,v in c.items()}
    if 3 in c.keys():
        return c[3]
    if 2 in c.keys():
        return c[2]

    return 'NEU'
    

stance_df['gpt_stance_aggregated'] = stance_df['gpt_stance'].apply(aggregate_stance)
stance_df['gpt_stance_aggregated_certain_NO'] = stance_df['gpt_stance'].apply(aggregate_stance_certain_NO)
stance_df.head()

,qid,query,docno,score,title_text,html_plain,rank,truncated_html_plain,gpt_stance,gpt_stance_aggregated,gpt_stance_aggregated_certain_NO
0,1,should teacher get tenure,clueweb22-en0031-49-02531,2800.1940,Pro & Con Quotes: Should Teachers Get Tenure? ...,Last updated on: 1/13/2011 | Author: ProCon.or...,0,Last updated on: 1/13/2011 | Author: ProCon.or...,"(NO, NEU, PRO)",NO,NEU
1,1,should teacher get tenure,clueweb22-en0042-24-00769,1911.2747,Teacher Tenure - Pros & Cons - ProCon.org,Last updated on: 1/13/2011 | Author: ProCon.or...,1,Last updated on: 1/13/2011 | Author: ProCon.or...,"(NEU, PRO, NEU)",NEU,NEU
2,1,should teacher get tenure,clueweb22-en0011-57-13248,1896.7843,Teacher Tenure Laws | LegalMatch,Most states have laws that protect public scho...,2,Most states have laws that protect public scho...,"(NEU, NO, NO)",NO,NO
3,1,should teacher get tenure,clueweb22-en0038-04-18313,1813.0868,What Is Teacher Tenure? | Education.com,"Charter schools, merit pay, vouchers, oh my! T...",3,"Charter schools, merit pay, vouchers, oh my! T...","(PRO, PRO, PRO)",PRO,PRO
4,1,should teacher get tenure,clueweb22-en0022-88-08363,1807.3335,Tenure | American Federation of Teachers,Tenure\n\nShare This\nPrint\n\nHow Due Process...,4,Tenure\n\nShare This\nPrint\n\nHow Due Process...,"(PRO, PRO, PRO)",PRO,PRO


In [4]:
def add_stance_to_df(df:pd.DataFrame, df_stance:pd.DataFrame) -> pd.DataFrame:
    """
    Add the aggregated stance to the dataframe.
    """
    assert len(df) == len(df_stance), (len(df), len(df_stance))

    # assert that dfs are the same
    for key in set(df_stance.keys()).intersection(set(df.keys())):
        if key in ['gpt_stance', 'gpt_stance_aggregated', 'gpt_stance_aggregated_certain_NO']:
            continue
        assert (df_stance[key] == df[key]).all(), (key, df_stance[key], df[key])

    df_out = df.copy()
    df_out['stance'] = df_stance['gpt_stance_aggregated']
    df_out['stance_certain_NO'] = df_stance['gpt_stance_aggregated_certain_NO']
    return df_out

# Quality Predictions

In [5]:
# NOTE: the ranks assigned by ChatNoir though seem to be in the ascending order, they are not consistent, i.e., some values are skipped sometimes, 
# some ranks start with 0, some with 1, so the best option is to do sorting by the retrieval score

In [5]:
import pandas as pd
import ast

meta_df = pd.read_csv("../../results-test/quality/chatnoir_10_custom_stopw_lemmas_meta_predictions.tsv", sep="\t")
meta_df['meta_prob'] = meta_df['meta_prob'].apply(ast.literal_eval)
meta_df = add_stance_to_df(meta_df, stance_df)
meta_df.head()

,qid,query,docno,score,title_text,html_plain,rank,meta_pred,meta_prob,stance,stance_certain_NO
0,1,should teacher get tenure,clueweb22-en0031-49-02531,2800.1940,Pro & Con Quotes: Should Teachers Get Tenure? ...,Last updated on: 1/13/2011 | Author: ProCon.or...,0,2,"[0.06334066002722948, 0.37788012162728407, 0.5...",NO,NEU
1,1,should teacher get tenure,clueweb22-en0042-24-00769,1911.2747,Teacher Tenure - Pros & Cons - ProCon.org,Last updated on: 1/13/2011 | Author: ProCon.or...,1,2,"[0.069465530539878, 0.401234753292063, 0.52929...",NEU,NEU
2,1,should teacher get tenure,clueweb22-en0011-57-13248,1896.7843,Teacher Tenure Laws | LegalMatch,Most states have laws that protect public scho...,2,2,"[0.15726173316843475, 0.3300798248400209, 0.51...",NO,NO
3,1,should teacher get tenure,clueweb22-en0038-04-18313,1813.0868,What Is Teacher Tenure? | Education.com,"Charter schools, merit pay, vouchers, oh my! T...",3,2,"[0.19096850485133235, 0.38419758020383077, 0.4...",PRO,PRO
4,1,should teacher get tenure,clueweb22-en0022-88-08363,1807.3335,Tenure | American Federation of Teachers,Tenure\n\nShare This\nPrint\n\nHow Due Process...,4,2,"[0.08173260553603098, 0.40576395373197, 0.5125...",PRO,PRO


In [6]:
fn = '../../results-test/quality/chatnoir_10_custom_stopw_lemmas_chatgpt.tsv'
gpt_df = pd.read_csv(fn, sep='\t')
gpt_df['gpt_quality'] = gpt_df['gpt_quality'].apply(ast.literal_eval)
gpt_df = add_stance_to_df(gpt_df, stance_df)
gpt_df.head()

,qid,query,docno,score,title_text,html_plain,rank,truncated_html_plain,gpt_quality,stance,stance_certain_NO
0,1,should teacher get tenure,clueweb22-en0031-49-02531,2800.1940,Pro & Con Quotes: Should Teachers Get Tenure? ...,Last updated on: 1/13/2011 | Author: ProCon.or...,0,Last updated on: 1/13/2011 | Author: ProCon.or...,"(2, 2, 2)",NO,NEU
1,1,should teacher get tenure,clueweb22-en0042-24-00769,1911.2747,Teacher Tenure - Pros & Cons - ProCon.org,Last updated on: 1/13/2011 | Author: ProCon.or...,1,Last updated on: 1/13/2011 | Author: ProCon.or...,"(2, 1, 1)",NEU,NEU
2,1,should teacher get tenure,clueweb22-en0011-57-13248,1896.7843,Teacher Tenure Laws | LegalMatch,Most states have laws that protect public scho...,2,Most states have laws that protect public scho...,"(2, 2, 2)",NO,NO
3,1,should teacher get tenure,clueweb22-en0038-04-18313,1813.0868,What Is Teacher Tenure? | Education.com,"Charter schools, merit pay, vouchers, oh my! T...",3,"Charter schools, merit pay, vouchers, oh my! T...","(1, 2, 2)",PRO,PRO
4,1,should teacher get tenure,clueweb22-en0022-88-08363,1807.3335,Tenure | American Federation of Teachers,Tenure\n\nShare This\nPrint\n\nHow Due Process...,4,Tenure\n\nShare This\nPrint\n\nHow Due Process...,"(1, 2, 2)",PRO,PRO


# Run 1

sort top-10 docs by the predicted quality label, within eqal labels sort by the predicted probability (rounded to 2 digits), if ties, sort by the chatnoir score 

In [9]:
def get_prob(row):
    return round(row["meta_prob"][row["meta_pred"]], 2)

run1 = meta_df.copy()
run1["one_prob"] = run1.apply(get_prob, axis=1) # fetch the probability of predicting the respective quality label
run1 = run1.sort_values(['qid', 'meta_pred', 'one_prob', 'score'], ascending=[True, False, False, False]) # sort as described in the run description
run1["rank"] = list(range(1,11))*50 # create hte ranks after the sorting (ascending order 1 ... 10)
run1["new_score"] = list(range(10,0,-1))*50 # create the scores after the sorting (descending order)
# run1["Q0"] = ["Q0"] * 500 # if the stance is not predicted, always "Q0", we can merge with the stance labels later
run1["Q0"] = run1['stance']
run1["tag"] = ["renji_abarai_meta_qual_prob"] * 500 # descriptive run tag

run1 = run1[["qid", "Q0", "docno", "rank", "new_score", "tag"]] # select the columns for the final run file

run1.to_csv("../../results-test/runs/run1.txt", sep=' ', index=False, header=None) # save the run file in the correct format

# Run 2

sort top-10 docs by the predicted quality label, within the eqal labels sort by the chatnoir score 

In [11]:
run2 = meta_df.copy()
run2 = run2.sort_values(['qid', 'meta_pred', 'score'], ascending=[True, False, False])
run2["rank"] = list(range(1,11))*50
run2["new_score"] = list(range(10,0,-1))*50
run2["Q0"] = run2['stance']
run2["tag"] = ["renji_abarai_meta_qual_score"] * 500

run2 = run2[["qid", "Q0", "docno", "rank", "new_score", "tag"]]

run2.to_csv("../../results-test/runs/run2.txt", sep=' ', index=False, header=None)

# Run 3

ChatGPT predictions for quality and for stance. 

two times neutral is rated the same as once low and once high. 

In [23]:
run3 = gpt_df.copy()

def aggregate_quality(quals:tuple[int]) -> float:
    """
    Aggregate quality labels (2,1,0) by taking the mean.
    """
    return sum(quals) / len(quals)

run3['gpt_quality_aggregated'] = run3['gpt_quality'].apply(aggregate_quality)

run3 = run3.sort_values(['qid', 'gpt_quality_aggregated', 'score'], ascending=[True, False, False])
run3["rank"] = list(range(1,11))*50
run3["new_score"] = list(range(10,0,-1))*50
run3["Q0"] = run3['stance']
run3["tag"] = ["renji_abarai_ChatGPT_mmEQhl"] * 500
run3.head()


,qid,query,docno,score,title_text,html_plain,rank,truncated_html_plain,gpt_quality,stance,gpt_quality_aggregated,new_score,Q0,tag
0,1,should teacher get tenure,clueweb22-en0031-49-02531,2800.1940,Pro & Con Quotes: Should Teachers Get Tenure? ...,Last updated on: 1/13/2011 | Author: ProCon.or...,1,Last updated on: 1/13/2011 | Author: ProCon.or...,"(2, 2, 2)",NO,2.0,10,NO,renji_abarai_ChatGPT_mmEQhl
2,1,should teacher get tenure,clueweb22-en0011-57-13248,1896.7843,Teacher Tenure Laws | LegalMatch,Most states have laws that protect public scho...,2,Most states have laws that protect public scho...,"(2, 2, 2)",NO,2.0,9,NO,renji_abarai_ChatGPT_mmEQhl
7,1,should teacher get tenure,clueweb22-en0033-93-00471,1521.0442,New York Teacher Tenure Rights - Horton Law PL...,Home » New York Management Law Blog » New York...,3,Home » New York Management Law Blog » New York...,"(2, 2, 2)",NO,2.0,8,NO,renji_abarai_ChatGPT_mmEQhl
8,1,should teacher get tenure,clueweb22-en0020-35-13458,1458.4889,Do you think teachers should get paid more? | ...,Do you think teachers should get paid more? | ...,4,Do you think teachers should get paid more? | ...,"(2, 2, 2)",NO,2.0,7,NO,renji_abarai_ChatGPT_mmEQhl
9,1,should teacher get tenure,clueweb22-en0031-66-06129,1455.3568,Tenure,Tenure Skip to main content\nFull Menu\n\nTenu...,5,Tenure Skip to main content\nFull Menu\n\nTenu...,"(2, 2, 2)",NO,2.0,6,NO,renji_abarai_ChatGPT_mmEQhl


In [24]:
run3 = run3[["qid", "Q0", "docno", "rank", "new_score", "tag"]]

run3.to_csv("../../results-test/runs/run3.txt", sep=' ', index=False, header=None)

# Run 4

ChatGPT for quality and stance. 

two times medium is rated higher than once high and once low. 

In [25]:
run4 = gpt_df.copy()

def aggregate_quality(quals:tuple[int]) -> float:
    """
    Aggregate quality labels (2,1,0) by taking the mean.
    """
    quals = [q + (0.1 if q==1 else 0) for q in quals]
    return sum(quals) / len(quals)

run4['gpt_quality_aggregated'] = run4['gpt_quality'].apply(aggregate_quality)

run4 = run4.sort_values(['qid', 'gpt_quality_aggregated', 'score'], ascending=[True, False, False])
run4["rank"] = list(range(1,11))*50
run4["new_score"] = list(range(10,0,-1))*50
run4["Q0"] = run4['stance']
run4["tag"] = ["renji_abarai_ChatGPT_mmGhl"] * 500
run4.head()


,qid,query,docno,score,title_text,html_plain,rank,truncated_html_plain,gpt_quality,stance,gpt_quality_aggregated,new_score,Q0,tag
0,1,should teacher get tenure,clueweb22-en0031-49-02531,2800.1940,Pro & Con Quotes: Should Teachers Get Tenure? ...,Last updated on: 1/13/2011 | Author: ProCon.or...,1,Last updated on: 1/13/2011 | Author: ProCon.or...,"(2, 2, 2)",NO,2.0,10,NO,renji_abarai_ChatGPT_mmGhl
2,1,should teacher get tenure,clueweb22-en0011-57-13248,1896.7843,Teacher Tenure Laws | LegalMatch,Most states have laws that protect public scho...,2,Most states have laws that protect public scho...,"(2, 2, 2)",NO,2.0,9,NO,renji_abarai_ChatGPT_mmGhl
7,1,should teacher get tenure,clueweb22-en0033-93-00471,1521.0442,New York Teacher Tenure Rights - Horton Law PL...,Home » New York Management Law Blog » New York...,3,Home » New York Management Law Blog » New York...,"(2, 2, 2)",NO,2.0,8,NO,renji_abarai_ChatGPT_mmGhl
8,1,should teacher get tenure,clueweb22-en0020-35-13458,1458.4889,Do you think teachers should get paid more? | ...,Do you think teachers should get paid more? | ...,4,Do you think teachers should get paid more? | ...,"(2, 2, 2)",NO,2.0,7,NO,renji_abarai_ChatGPT_mmGhl
9,1,should teacher get tenure,clueweb22-en0031-66-06129,1455.3568,Tenure,Tenure Skip to main content\nFull Menu\n\nTenu...,5,Tenure Skip to main content\nFull Menu\n\nTenu...,"(2, 2, 2)",NO,2.0,6,NO,renji_abarai_ChatGPT_mmGhl


In [26]:
run4 = run4[["qid", "Q0", "docno", "rank", "new_score", "tag"]]

run4.to_csv("../../results-test/runs/run4.txt", sep=' ', index=False, header=None)

# Run 5

First rank documents with stance 'NO' to the bottom, then rank by ChatGPT quality. 

two mediums is rated equal to one high and one low

In [31]:
run5 = gpt_df.copy()

def aggregate_quality(quals:tuple[int]) -> float:
    """
    Aggregate quality labels (2,1,0) by taking the mean.
    """
    return sum(quals) / len(quals)

def stance_to_value(stance:str):
    if stance == 'NO':
        return 0
    return 1

run5['gpt_quality_aggregated'] = run5['gpt_quality'].apply(aggregate_quality)
run5['stance_quality'] = run5['stance'].apply(stance_to_value)


run5 = run5.sort_values(['qid', 'stance_quality', 'gpt_quality_aggregated', 'score'], ascending=[True, False, False, False])
run5["rank"] = list(range(1,11))*50
run5["new_score"] = list(range(10,0,-1))*50
run5["Q0"] = run5['stance']
run5["tag"] = ["renji_abarai_stance_ChatGPT"] * 500
run5[:10]


,qid,query,docno,score,title_text,html_plain,rank,truncated_html_plain,gpt_quality,stance,gpt_quality_aggregated,stance_quality,new_score,Q0,tag
3,1,should teacher get tenure,clueweb22-en0038-04-18313,1813.0868,What Is Teacher Tenure? | Education.com,"Charter schools, merit pay, vouchers, oh my! T...",1,"Charter schools, merit pay, vouchers, oh my! T...","(1, 2, 2)",PRO,1.666667,1,10,PRO,renji_abarai_stance_ChatGPT
4,1,should teacher get tenure,clueweb22-en0022-88-08363,1807.3335,Tenure | American Federation of Teachers,Tenure\n\nShare This\nPrint\n\nHow Due Process...,2,Tenure\n\nShare This\nPrint\n\nHow Due Process...,"(1, 2, 2)",PRO,1.666667,1,9,PRO,renji_abarai_stance_ChatGPT
1,1,should teacher get tenure,clueweb22-en0042-24-00769,1911.2747,Teacher Tenure - Pros & Cons - ProCon.org,Last updated on: 1/13/2011 | Author: ProCon.or...,3,Last updated on: 1/13/2011 | Author: ProCon.or...,"(2, 1, 1)",NEU,1.333333,1,8,NEU,renji_abarai_stance_ChatGPT
5,1,should teacher get tenure,clueweb22-en0021-01-08109,1660.7153,Argumentative Essay: Should Teachers Get Paid?...,Argumentative Essay: Should Teachers Get Paid?...,4,Argumentative Essay: Should Teachers Get Paid?...,"(2, 1, 1)",PRO,1.333333,1,7,PRO,renji_abarai_stance_ChatGPT
6,1,should teacher get tenure,clueweb22-en0032-04-03343,1553.3670,Teachers and Tenure: Both Sides of the Heated ...,Teachers and Tenure: Both Sides of the Heated ...,5,Teachers and Tenure: Both Sides of the Heated ...,"(1, 1, 1)",NEU,1.000000,1,6,NEU,renji_abarai_stance_ChatGPT
0,1,should teacher get tenure,clueweb22-en0031-49-02531,2800.1940,Pro & Con Quotes: Should Teachers Get Tenure? ...,Last updated on: 1/13/2011 | Author: ProCon.or...,6,Last updated on: 1/13/2011 | Author: ProCon.or...,"(2, 2, 2)",NO,2.000000,0,5,NO,renji_abarai_stance_ChatGPT
2,1,should teacher get tenure,clueweb22-en0011-57-13248,1896.7843,Teacher Tenure Laws | LegalMatch,Most states have laws that protect public scho...,7,Most states have laws that protect public scho...,"(2, 2, 2)",NO,2.000000,0,4,NO,renji_abarai_stance_ChatGPT
7,1,should teacher get tenure,clueweb22-en0033-93-00471,1521.0442,New York Teacher Tenure Rights - Horton Law PL...,Home » New York Management Law Blog » New York...,8,Home » New York Management Law Blog » New York...,"(2, 2, 2)",NO,2.000000,0,3,NO,renji_abarai_stance_ChatGPT
8,1,should teacher get tenure,clueweb22-en0020-35-13458,1458.4889,Do you think teachers should get paid more? | ...,Do you think teachers should get paid more? | ...,9,Do you think teachers should get paid more? | ...,"(2, 2, 2)",NO,2.000000,0,2,NO,renji_abarai_stance_ChatGPT
9,1,should teacher get tenure,clueweb22-en0031-66-06129,1455.3568,Tenure,Tenure Skip to main content\nFull Menu\n\nTenu...,10,Tenure Skip to main content\nFull Menu\n\nTenu...,"(2, 2, 2)",NO,2.000000,0,1,NO,renji_abarai_stance_ChatGPT


In [32]:
run5 = run5[["qid", "Q0", "docno", "rank", "new_score", "tag"]]

run5.to_csv("../../results-test/runs/run5.txt", sep=' ', index=False, header=None)

# Run 6
same as run 5 (ranked by stance and quality) but the NO stance is only considered when it was predicted by majority of the ChatGPT stance runs

In [43]:
run6 = gpt_df.copy()

def aggregate_quality(quals:tuple[int]) -> float:
    """
    Aggregate quality labels (2,1,0) by taking the mean.
    """
    return sum(quals) / len(quals)

def stance_to_value(stance:str):
    if stance == 'NO':
        return 0
    return 1

run6['gpt_quality_aggregated'] = run6['gpt_quality'].apply(aggregate_quality)
run6['stance_quality'] = run6['stance_certain_NO'].apply(stance_to_value)


run6 = run6.sort_values(['qid', 'stance_quality', 'gpt_quality_aggregated', 'score'], ascending=[True, False, False, False])
run6["rank"] = list(range(1,11))*50
run6["new_score"] = list(range(10,0,-1))*50
run6["Q0"] = run6['stance']
run6["tag"] = ["renji_abarai_stance-certainNO_ChatGPT"] * 500
run6[:10]


,qid,query,docno,score,title_text,html_plain,rank,truncated_html_plain,gpt_quality,stance,stance_certain_NO,gpt_quality_aggregated,stance_quality,new_score,Q0,tag
0,1,should teacher get tenure,clueweb22-en0031-49-02531,2800.1940,Pro & Con Quotes: Should Teachers Get Tenure? ...,Last updated on: 1/13/2011 | Author: ProCon.or...,1,Last updated on: 1/13/2011 | Author: ProCon.or...,"(2, 2, 2)",NO,NEU,2.000000,1,10,NO,renji_abarai_stance-certainNO_ChatGPT
3,1,should teacher get tenure,clueweb22-en0038-04-18313,1813.0868,What Is Teacher Tenure? | Education.com,"Charter schools, merit pay, vouchers, oh my! T...",2,"Charter schools, merit pay, vouchers, oh my! T...","(1, 2, 2)",PRO,PRO,1.666667,1,9,PRO,renji_abarai_stance-certainNO_ChatGPT
4,1,should teacher get tenure,clueweb22-en0022-88-08363,1807.3335,Tenure | American Federation of Teachers,Tenure\n\nShare This\nPrint\n\nHow Due Process...,3,Tenure\n\nShare This\nPrint\n\nHow Due Process...,"(1, 2, 2)",PRO,PRO,1.666667,1,8,PRO,renji_abarai_stance-certainNO_ChatGPT
1,1,should teacher get tenure,clueweb22-en0042-24-00769,1911.2747,Teacher Tenure - Pros & Cons - ProCon.org,Last updated on: 1/13/2011 | Author: ProCon.or...,4,Last updated on: 1/13/2011 | Author: ProCon.or...,"(2, 1, 1)",NEU,NEU,1.333333,1,7,NEU,renji_abarai_stance-certainNO_ChatGPT
5,1,should teacher get tenure,clueweb22-en0021-01-08109,1660.7153,Argumentative Essay: Should Teachers Get Paid?...,Argumentative Essay: Should Teachers Get Paid?...,5,Argumentative Essay: Should Teachers Get Paid?...,"(2, 1, 1)",PRO,PRO,1.333333,1,6,PRO,renji_abarai_stance-certainNO_ChatGPT
6,1,should teacher get tenure,clueweb22-en0032-04-03343,1553.3670,Teachers and Tenure: Both Sides of the Heated ...,Teachers and Tenure: Both Sides of the Heated ...,6,Teachers and Tenure: Both Sides of the Heated ...,"(1, 1, 1)",NEU,NEU,1.000000,1,5,NEU,renji_abarai_stance-certainNO_ChatGPT
2,1,should teacher get tenure,clueweb22-en0011-57-13248,1896.7843,Teacher Tenure Laws | LegalMatch,Most states have laws that protect public scho...,7,Most states have laws that protect public scho...,"(2, 2, 2)",NO,NO,2.000000,0,4,NO,renji_abarai_stance-certainNO_ChatGPT
7,1,should teacher get tenure,clueweb22-en0033-93-00471,1521.0442,New York Teacher Tenure Rights - Horton Law PL...,Home » New York Management Law Blog » New York...,8,Home » New York Management Law Blog » New York...,"(2, 2, 2)",NO,NO,2.000000,0,3,NO,renji_abarai_stance-certainNO_ChatGPT
8,1,should teacher get tenure,clueweb22-en0020-35-13458,1458.4889,Do you think teachers should get paid more? | ...,Do you think teachers should get paid more? | ...,9,Do you think teachers should get paid more? | ...,"(2, 2, 2)",NO,NO,2.000000,0,2,NO,renji_abarai_stance-certainNO_ChatGPT
9,1,should teacher get tenure,clueweb22-en0031-66-06129,1455.3568,Tenure,Tenure Skip to main content\nFull Menu\n\nTenu...,10,Tenure Skip to main content\nFull Menu\n\nTenu...,"(2, 2, 2)",NO,NO,2.000000,0,1,NO,renji_abarai_stance-certainNO_ChatGPT


In [44]:
run6 = run6[["qid", "Q0", "docno", "rank", "new_score", "tag"]]

run6.to_csv("../../results-test/runs/run6.txt", sep=' ', index=False, header=None)

# Run 7

baseline run, takes the ChatNoir ranking but the queries are processed with lemmatiaztion and customized stop-word removal

In [8]:
import pandas as pd
import ast

meta_df = pd.read_csv("../../results-test/quality/chatnoir_10_custom_stopw_lemmas_meta_predictions.tsv", sep="\t")
#meta_df['meta_prob'] = meta_df['meta_prob'].apply(ast.literal_eval)
meta_df = add_stance_to_df(meta_df, stance_df)
meta_df.head()

,qid,query,docno,score,title_text,html_plain,rank,meta_pred,meta_prob,stance,stance_certain_NO
0,1,should teacher get tenure,clueweb22-en0031-49-02531,2800.1940,Pro & Con Quotes: Should Teachers Get Tenure? ...,Last updated on: 1/13/2011 | Author: ProCon.or...,0,2,"[0.06334066002722948, 0.37788012162728407, 0.5...",NO,NEU
1,1,should teacher get tenure,clueweb22-en0042-24-00769,1911.2747,Teacher Tenure - Pros & Cons - ProCon.org,Last updated on: 1/13/2011 | Author: ProCon.or...,1,2,"[0.069465530539878, 0.401234753292063, 0.52929...",NEU,NEU
2,1,should teacher get tenure,clueweb22-en0011-57-13248,1896.7843,Teacher Tenure Laws | LegalMatch,Most states have laws that protect public scho...,2,2,"[0.15726173316843475, 0.3300798248400209, 0.51...",NO,NO
3,1,should teacher get tenure,clueweb22-en0038-04-18313,1813.0868,What Is Teacher Tenure? | Education.com,"Charter schools, merit pay, vouchers, oh my! T...",3,2,"[0.19096850485133235, 0.38419758020383077, 0.4...",PRO,PRO
4,1,should teacher get tenure,clueweb22-en0022-88-08363,1807.3335,Tenure | American Federation of Teachers,Tenure\n\nShare This\nPrint\n\nHow Due Process...,4,2,"[0.08173260553603098, 0.40576395373197, 0.5125...",PRO,PRO


In [10]:
run7 = meta_df.copy()

run7["rank"] = list(range(1,11))*50 # create hte ranks after the sorting (ascending order 1 ... 10)
run7["new_score"] = list(range(10,0,-1))*50 # create the scores after the sorting (descending order)
run7["Q0"] = run7['stance']
run7["tag"] = ["renji_abarai_baseline"] * 500 # descriptive run tag

run7 = run7[["qid", "Q0", "docno", "rank", "new_score", "tag"]] # select the columns for the final run file

run7.to_csv("../../results-test/runs/run7.txt", sep=' ', index=False, header=None) # save the run file in the correct format

# No Run

This is not an official run. I only use it to compute the stance macro F1 in the certainNO setting

In [7]:
run8 = gpt_df.copy()

def aggregate_quality(quals:tuple[int]) -> float:
    """
    Aggregate quality labels (2,1,0) by taking the mean.
    """
    return sum(quals) / len(quals)

def stance_to_value(stance:str):
    if stance == 'NO':
        return 0
    return 1

run8['gpt_quality_aggregated'] = run8['gpt_quality'].apply(aggregate_quality)
run8['stance_quality'] = run8['stance_certain_NO'].apply(stance_to_value)


run8 = run8.sort_values(['qid', 'stance_quality', 'gpt_quality_aggregated', 'score'], ascending=[True, False, False, False])
run8["rank"] = list(range(1,11))*50
run8["new_score"] = list(range(10,0,-1))*50
run8["Q0"] = run8['stance_certain_NO']
run8["tag"] = ["fake_renji_abarai_stance-certainNO_ChatGPT"] * 500
run8[:10]


,qid,query,docno,score,title_text,html_plain,rank,truncated_html_plain,gpt_quality,stance,stance_certain_NO,gpt_quality_aggregated,stance_quality,new_score,Q0,tag
0,1,should teacher get tenure,clueweb22-en0031-49-02531,2800.1940,Pro & Con Quotes: Should Teachers Get Tenure? ...,Last updated on: 1/13/2011 | Author: ProCon.or...,1,Last updated on: 1/13/2011 | Author: ProCon.or...,"(2, 2, 2)",NO,NEU,2.000000,1,10,NEU,fake_renji_abarai_stance-certainNO_ChatGPT
3,1,should teacher get tenure,clueweb22-en0038-04-18313,1813.0868,What Is Teacher Tenure? | Education.com,"Charter schools, merit pay, vouchers, oh my! T...",2,"Charter schools, merit pay, vouchers, oh my! T...","(1, 2, 2)",PRO,PRO,1.666667,1,9,PRO,fake_renji_abarai_stance-certainNO_ChatGPT
4,1,should teacher get tenure,clueweb22-en0022-88-08363,1807.3335,Tenure | American Federation of Teachers,Tenure\n\nShare This\nPrint\n\nHow Due Process...,3,Tenure\n\nShare This\nPrint\n\nHow Due Process...,"(1, 2, 2)",PRO,PRO,1.666667,1,8,PRO,fake_renji_abarai_stance-certainNO_ChatGPT
1,1,should teacher get tenure,clueweb22-en0042-24-00769,1911.2747,Teacher Tenure - Pros & Cons - ProCon.org,Last updated on: 1/13/2011 | Author: ProCon.or...,4,Last updated on: 1/13/2011 | Author: ProCon.or...,"(2, 1, 1)",NEU,NEU,1.333333,1,7,NEU,fake_renji_abarai_stance-certainNO_ChatGPT
5,1,should teacher get tenure,clueweb22-en0021-01-08109,1660.7153,Argumentative Essay: Should Teachers Get Paid?...,Argumentative Essay: Should Teachers Get Paid?...,5,Argumentative Essay: Should Teachers Get Paid?...,"(2, 1, 1)",PRO,PRO,1.333333,1,6,PRO,fake_renji_abarai_stance-certainNO_ChatGPT
6,1,should teacher get tenure,clueweb22-en0032-04-03343,1553.3670,Teachers and Tenure: Both Sides of the Heated ...,Teachers and Tenure: Both Sides of the Heated ...,6,Teachers and Tenure: Both Sides of the Heated ...,"(1, 1, 1)",NEU,NEU,1.000000,1,5,NEU,fake_renji_abarai_stance-certainNO_ChatGPT
2,1,should teacher get tenure,clueweb22-en0011-57-13248,1896.7843,Teacher Tenure Laws | LegalMatch,Most states have laws that protect public scho...,7,Most states have laws that protect public scho...,"(2, 2, 2)",NO,NO,2.000000,0,4,NO,fake_renji_abarai_stance-certainNO_ChatGPT
7,1,should teacher get tenure,clueweb22-en0033-93-00471,1521.0442,New York Teacher Tenure Rights - Horton Law PL...,Home » New York Management Law Blog » New York...,8,Home » New York Management Law Blog » New York...,"(2, 2, 2)",NO,NO,2.000000,0,3,NO,fake_renji_abarai_stance-certainNO_ChatGPT
8,1,should teacher get tenure,clueweb22-en0020-35-13458,1458.4889,Do you think teachers should get paid more? | ...,Do you think teachers should get paid more? | ...,9,Do you think teachers should get paid more? | ...,"(2, 2, 2)",NO,NO,2.000000,0,2,NO,fake_renji_abarai_stance-certainNO_ChatGPT
9,1,should teacher get tenure,clueweb22-en0031-66-06129,1455.3568,Tenure,Tenure Skip to main content\nFull Menu\n\nTenu...,10,Tenure Skip to main content\nFull Menu\n\nTenu...,"(2, 2, 2)",NO,NO,2.000000,0,1,NO,fake_renji_abarai_stance-certainNO_ChatGPT


In [ ]:
run8 = run8[["qid", "Q0", "docno", "rank", "new_score", "tag"]]

run8.to_csv("../../results-test/runs/fake_run8.txt", sep=' ', index=False, header=None)